## Setup notebook

Do all imports.

In [ ]:
# For numerical methods
import numpy as np
from scipy.spatial.transform import Rotation
from scipy.linalg import block_diag

# For image processing and visualization of results
import cv2
import matplotlib.pyplot as plt
from matplotlib.patches import ConnectionPatch

# For timing
import time

Create random number generator with a particular seed so we can reproduce results.

In [ ]:
rng = np.random.default_rng(0)

Define a function that implements the wedge operator.

In [ ]:
def skew(v):
    assert(type(v) == np.ndarray)
    assert(v.shape == (3,))
    return np.array([[0., -v[2], v[1]],
                     [v[2], 0., -v[0]],
                     [-v[1], v[0], 0.]])

Define a function to print things nicely.

In [ ]:
def myprint(M):
    if M.shape:
        with np.printoptions(linewidth=150, formatter={'float': lambda x: f'{x:10.4f}'}):
            print(M)
    else:
        print(f'{M:10.4f}')

## Create dataset

Choose intrinsic parameters.

In [ ]:
K = np.array([
    [1500., 0., 1000.],
    [0., 1500., 500.],
    [0., 0., 1.],
])

Choose extrinsic parameters (these are poses **of the camera in the world frame**).

In [ ]:
R1 = np.eye(3)
p1 = np.array([0.0, 0.0, -1.0])
T1 = np.row_stack([np.column_stack([R1, p1]), [0., 0., 0., 1.]])

R2 = Rotation.from_rotvec((0.05 * np.pi) * np.array([0., 1., 0.])).as_matrix()
p2 = np.array([0.5, 0.0, -1.1])
T2 = np.row_stack([np.column_stack([R2, p2]), [0., 0., 0., 1.]])

Sample points **in the world frame**.

In [ ]:
n = 10
p_inW = rng.uniform(low=[-1., -1., -0.5], high=[1., 1., 2.5], size=(n, 3))

Find coordinates of these same points **in the frame of camera 1** for later comparison.

In [ ]:
p_in1 = []
for p in p_inW:
    p_in1.append(R1.T @ p - R1.T @ p1)
p_in1 = np.array(p_in1)

Project points into the images.

In [ ]:
def project(p_inW, K, T_inW_ofC):
    p_inW_homog = np.concatenate([p_inW, [1.]])
    p_inC_homog = np.linalg.inv(T_inW_ofC) @ p_inW_homog
    assert(p_inC_homog[2] > 0)
    q_homog = K @ np.eye(3, 4) @ p_inC_homog
    return (q_homog / q_homog[2])[:-1]

q1 = []
q2 = []
for p in p_inW:
    q1.append(project(p, K, T1))
    q2.append(project(p, K, T2))
q1 = np.array(q1)
q2 = np.array(q2)

Get pose $T^2_1$ of camera 2 in the coordinates of camera 1.

In [ ]:
T_in2_of1 = np.linalg.inv(T2) @ T1
myprint(T_in2_of1)

## Get reference solution with OpenCV

Estimate the essential matrix $E$ and the relative pose $(R_1^2, p_1^2)$, where $p_1^2$ is correct only up to scale.

In [ ]:
num_inliers_cv, E_cv, R_cv, p_cv, mask_cv = cv2.recoverPose(
    q1,
    q2,
    K, np.zeros(4),
    K, np.zeros(4),
)

p_cv = p_cv.flatten()

T_in2_of1_cv = np.row_stack([np.column_stack([R_cv, p_cv]), [0., 0., 0., 1.]])
print('T_in2_of1 =')
myprint(T_in2_of1_cv)

print('\nIs relative position (aka translation) the same up to scale?')
myprint(T_in2_of1[0:3, 3] / np.linalg.norm(T_in2_of1[0:3, 3]))
myprint(p_cv / np.linalg.norm(p_cv))

## Implement solution yourself

Implement a function to find structure (i.e., do triangulation).

In [ ]:
def triangulate(q1, q2, R, p):
    # Return two arrays of size n x 3
    # - the first should have coordinates of all points in the frame of camera 1
    # - the second should have coordinates of all points in the frame of camera 2
    pass

Implement a function to do two-view reconstruction:
* Find the essential matrix
* Decompose the essential matrix to find the relative pose $(R,p)$ of camera 1 in camera 2
* Triangulate to find 3D coordinates of all points

In [ ]:
def twoview(q1, q2, K):
    # Return:
    # - the essential matrix (3x3)
    # - the relative pose R (3x3) and p (length 3)
    # - the coordinates of all points to which q1 and q2 correspond in the coordinates of camera 1
    pass

Apply your function and check that the results are correct.